In [ ]:
# Re-create files with safer quoting (outer triple double-quotes), avoiding syntax issues.
import os
from textwrap import dedent

app_code = """\
import os
import re
import pandas as pd
import streamlit as st
import psycopg2
import google.generativeai as genai

# ------------------------------
# Config & Setup
# ------------------------------
st.set_page_config(page_title="NL → SQL (Gemini) for Postgres", page_icon="🧠", layout="wide")

# Read environment variables (recommended: load from .env via `python-dotenv` or Streamlit secrets)
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY", "")
DB_HOST = os.getenv("DB_HOST", "localhost")
DB_PORT = int(os.getenv("DB_PORT", "5432"))
DB_NAME = os.getenv("DB_NAME", "postgres")
DB_USER = os.getenv("DB_USER", "postgres")
DB_PASSWORD = os.getenv("DB_PASSWORD", "")

if not GOOGLE_API_KEY:
    st.warning("Missing GOOGLE_API_KEY environment variable. Set it before running the app.")
else:
    genai.configure(api_key=GOOGLE_API_KEY)

# Model selection
DEFAULT_MODEL = "gemini-1.5-pro"  # change if you have access to other versions

# ------------------------------
# Utility: Database connection
# ------------------------------
@st.cache_resource(show_spinner=False)
def get_conn():
    return psycopg2.connect(
        host=DB_HOST, port=DB_PORT, dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD
    )

# ------------------------------
# Utility: Read-only schema snapshot
# ------------------------------
@st.cache_data(ttl=300, show_spinner=False)
def get_schema_snapshot(schema_filter=None):
    q = '''
    SELECT
        table_schema,
        table_name,
        column_name,
        data_type
    FROM information_schema.columns
    WHERE table_schema NOT IN ('pg_catalog','information_schema')
    ORDER BY table_schema, table_name, ordinal_position;
    '''
    if schema_filter:
        # naive filter: only include matching schema names (comma-separated list)
        allowed = tuple([s.strip() for s in schema_filter.split(",")])
    else:
        allowed = None

    with get_conn() as conn:
        df = pd.read_sql(q, conn)

    if allowed:
        df = df[df["table_schema"].isin(allowed)].copy()

    # Build compact schema text
    parts = []
    for (schema, table), g in df.groupby(["table_schema","table_name"]):
        cols = ", ".join([f"{r.column_name} ({r.data_type})" for r in g.itertuples(index=False)])
        parts.append(f"{schema}.{table}: {cols}")
    schema_text = "\\n".join(parts)
    return df, schema_text

# ------------------------------
# Prompt builder
# ------------------------------
SYSTEM_RULES = \"\"\"\
You are a senior data analyst that writes safe, syntactically-correct PostgreSQL for read-only analytics.
- Use only SELECT queries. Do NOT write INSERT/UPDATE/DELETE/ALTER/DROP/TRUNCATE/COPY/VACUUM or DDL.
- Do not use multiple statements; return exactly one SQL statement.
- Prefer explicit schema-qualified table names (schema.table).
- Use columns and tables that exist in the provided schema.
- If aggregation is requested, include GROUP BY as needed.
- If the user asks for "top" or "latest", include ORDER BY and LIMIT.
- If the request is ambiguous, make reasonable assumptions and add comments as inline SQL comments.
- Output SQL only, with no prose. If you include code fences, ensure they are triple backticks with 'sql'.
\"\"\"

def build_prompt(user_question, schema_text):
    return f\"\"\"\
{SYSTEM_RULES}

Here is the database schema (schema-qualified with types):
---
{schema_text}
---

Write a single PostgreSQL SELECT statement that answers the user's question.
Use CURRENT_DATE for "today" when needed.

User question:
\"\"\"{user_question}\"\"\"

Return only SQL (optionally inside ```sql code fences).
\"\"\"

# ------------------------------
# SQL safety checks
# ------------------------------
FORBIDDEN = re.compile(r\"\\b(INSERT|UPDATE|DELETE|ALTER|DROP|TRUNCATE|VACUUM|CREATE|GRANT|REVOKE|COPY|;|\\$\\$)\\b\", re.IGNORECASE)

def extract_sql(text):
    # try to pull from a ```sql ... ``` block first
    m = re.search(r\"```sql\\s*(.*?)```\", text, re.IGNORECASE | re.DOTALL)
    if m:
        return m.group(1).strip()
    # otherwise return the whole text
    return text.strip()

def enforce_read_only(sql):
    # forbid unsafe keywords and multiple statements
    if FORBIDDEN.search(sql):
        raise ValueError(\"Generated SQL contains forbidden or unsafe keywords.\")
    # quick check for multiple statements by semicolon in the middle
    if \";\" in sql.strip().rstrip(\";\"):
        raise ValueError(\"Multiple statements detected; only a single SELECT is allowed.\")
    if not re.match(r\"^\\s*SELECT\\b\", sql, re.IGNORECASE):
        raise ValueError(\"Only SELECT queries are allowed.\")
    # ensure a LIMIT if not present (for safety)
    if re.search(r\"\\bLIMIT\\b\", sql, re.IGNORECASE) is None:
        sql = sql.rstrip().rstrip(\";\") + \" LIMIT 100\"
    return sql

# ------------------------------
# Gemini call
# ------------------------------
def generate_sql(question, schema_text, temperature=0.2, model_name=DEFAULT_MODEL):
    prompt = build_prompt(question, schema_text)
    model = genai.GenerativeModel(model_name)
    resp = model.generate_content(prompt, generation_config=genai.types.GenerationConfig(
        temperature=temperature,
    ))
    return resp.text or \"\"

# ------------------------------
# UI
# ------------------------------
st.title(\"🧠 NL → SQL for Postgres (Gemini)\")
st.caption(\"Convert natural language to safe, read-only SQL and query your PostgreSQL database.\")

with st.expander(\"🔧 Configuration\", expanded=False):
    model_name = st.text_input(\"Gemini model\", value=DEFAULT_MODEL, help=\"e.g., gemini-1.5-pro, gemini-1.5-flash\")
    temperature = st.slider(\"Creativity (temperature)\", 0.0, 1.0, 0.2, 0.05)
    schema_filter = st.text_input(\"Schema filter (optional, comma-separated)\", value=\"\", help=\"e.g., public, analytics\")

    st.markdown(\"**Database connection (from environment)**\")
    col1, col2, col3 = st.columns(3)
    col1.text_input(\"DB_HOST\", value=DB_HOST, disabled=True)
    col2.text_input(\"DB_PORT\", value=str(DB_PORT), disabled=True)
    col3.text_input(\"DB_NAME\", value=DB_NAME, disabled=True)
    col4, col5 = st.columns(2)
    col4.text_input(\"DB_USER\", value=DB_USER, disabled=True)
    col5.text_input(\"DB_PASSWORD\", value=\"*****\", disabled=True)

schema_df, schema_text = get_schema_snapshot(schema_filter if schema_filter.strip() else None)

st.subheader(\"📚 Detected Schema\")
with st.expander(\"Show schema\", expanded=False):
    st.text(schema_text if schema_text else \"No tables found.\")

question = st.text_input(\"Ask a question about your data:\", placeholder=\"e.g., What are the top 10 products by sales last month?\")
run_btn = st.button(\"Generate SQL and Run\", type=\"primary\")

if run_btn and question.strip():
    with st.spinner(\"Thinking with Gemini and preparing SQL...\\"):
        try:
            llm_text = generate_sql(question.strip(), schema_text, temperature=temperature, model_name=model_name)
            raw_sql = extract_sql(llm_text)
            safe_sql = enforce_read_only(raw_sql)
        except Exception as e:
            st.error(f\"Failed to generate safe SQL: {e}\")
            if 'llm_text' in locals():
                with st.expander(\"LLM output (raw)\", expanded=False):
                    st.code(llm_text, language=\"markdown\")
            st.stop()

    st.markdown(\"#### 🧩 Generated SQL\")
    st.code(safe_sql, language=\"sql\")

    # Execute
    try:
        with st.spinner(\"Running query on PostgreSQL...\"):
            with get_conn() as conn:
                df = pd.read_sql(safe_sql, conn)
    except Exception as e:
        st.error(f\"Database error: {e}\")
        st.stop()

    # Show
    st.markdown(\"#### 📊 Results\")
    if df.empty:
        st.info(\"No rows returned.\")
    else:
        st.dataframe(df, use_container_width=True)
        # Offer CSV download
        csv = df.to_csv(index=False).encode(\"utf-8\")
        st.download_button(\"Download CSV\", csv, file_name=\"results.csv\", mime=\"text/csv\")

st.markdown(\"---\")
st.caption(\"Safety: enforces single-statement, SELECT-only queries and auto-adds LIMIT 100 when absent.\")
"""

requirements = """\
streamlit==1.37.1
pandas==2.2.2
psycopg2-binary==2.9.9
google-generativeai==0.7.2
python-dotenv==1.0.1
"""

env_example = """\
# Rename this file to .env and fill in values
GOOGLE_API_KEY=your_gemini_api_key_here

# PostgreSQL connection
DB_HOST=localhost
DB_PORT=5432
DB_NAME=postgres
DB_USER=postgres
DB_PASSWORD=your_password
"""

base = "/mnt/data"
os.makedirs(base, exist_ok=True)
with open(os.path.join(base, "app.py"), "w", encoding="utf-8") as f:
    f.write(app_code)

with open(os.path.join(base, "requirements.txt"), "w", encoding="utf-8") as f:
    f.write(requirements)

with open(os.path.join(base, ".env.example"), "w", encoding="utf-8") as f:
    f.write(env_example)

sorted(os.listdir(base))
